In [1]:
using Pkg;
Pkg.add("CSV")
Pkg.add("DataFrames")

using CSV
using DataFrames

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [70]:
file_path = "dataset/Android_Malware.csv"

data = CSV.File(file_path, header=true) |> DataFrame

columns_to_drop = ["Column1", "Flow ID", " Source IP", " Destination IP", " Timestamp"]

select!(data, Not(columns_to_drop))
dropmissing!(data)


Row,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String15,Float64,String7,Float64,Float64,Float64,Float64,String7,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String31
1,50004,443.0,6.0,37027,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0146,37027.0,0.0,37027.0,37027.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,32,32,27.0073,27.0073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,0.0,1.0,0.0,0.0,0.0,32.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2994.0,362.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
2,35455,443.0,6.0,36653,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.5658,36653.0,0.0,36653.0,36653.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,32,32,27.2829,27.2829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,0.0,1.0,0.0,0.0,0.0,32.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2860.0,362.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
3,51775,443.0,6.0,534099,8,12,1011.0,11924.0,581.0,0.0,126.375,207.799,1460.0,0.0,993.667,656.474,24218.4,37.4462,28110.5,43148.1,158676.0,8.0,481340.0,68762.9,62376.2,158676.0,449.0,487990.0,44362.7,86342.0,217790.0,8.0,0.0,0.0,0,0,180,244,14.9785,22.4677,0.0,1460.0,615.952,672.745,4.52586e5,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,1.0,646.75,126.375,993.667,180.0,0,0.0,0.0,0.0,0.0,0.0,8.0,1011.0,12.0,11924.0,65535.0,63441.0,3.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
4,51775,443.0,6.0,9309,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322.269,4654.5,5137.13,8287.0,1022.0,9309.0,4654.5,5137.13,8287.0,1022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,60,0,322.269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,60.0,0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,65535.0,-1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
5,51776,443.0,6.0,19890496,8,6,430.0,5679.0,218.0,0.0,53.75,99.5386,1460.0,0.0,946.5,710.412,307.132,0.703854,1.53004e6,5.37789e6,1.94267e7,7.0,1.98905e7,2.8415e6,7.31409e6,1.94267e7,639.0,410964.0,82192.8,1.54846e5,355945.0,7.0,0.0,0.0,0,0,180,124,0.402202,0.301652,0.0,1460.0,407.267,626.958,3.93077e5,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,436.357,53.75,946.5,180.0,0,0.0,0.0,0.0,0.0,0.0,8.0,430.0,6.0,5679.0,65535.0,64022.0,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [71]:
#show(data, allrows=false)
describe(data)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Source Port,39138.3,0,43226.0,65531,0,Int64
2,Destination Port,5198.78,0.0,443.0,65428.0,0,Float64
3,Protocol,8.33532,0.0,6.0,17.0,0,Float64
4,Flow Duration,1.09299e7,-1,560232.0,119999934,0,Int64
5,Total Fwd Packets,7.35741,1,2.0,17877,0,Int64
6,Total Backward Packets,10.4946,0,1.0,55652,0,Int64
7,Total Length of Fwd Packets,679.2,0.0,31.0,9.04378e6,0,Float64
8,Total Length of Bwd Packets,11294.2,0.0,26.0,8.05097e7,0,Float64
9,Fwd Packet Length Max,212.002,0.0,31.0,1460.0,0,Float64


### First approach: binary classification

In [72]:
function transform_binary_class(output_data)
    binary_labels = BitVector(output_data .== "Benign")
    return binary_labels
end

input_data = Matrix(data[!, 1:80]);

output_data = data[!, :Label];

binary_labels = transform_binary_class(output_data)
@assert binary_labels isa BitVector
@assert input_data isa Matrix

In [73]:
include("functions.jl")

train_indexes, test_indexes = holdOut(size(input_data, 1), 0.2)

train_input = input_data[train_indexes, :]
train_output = binary_labels[train_indexes, :]

test_input = input_data[test_indexes, :]
test_output = binary_labels[test_indexes, :]

@assert size(test_input, 1) == size(test_output, 1)
@assert size(train_input, 1) == size(train_output, 1)

In [74]:
using ScikitLearn

@sk_import svm:SVC
@sk_import tree:DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=4)

fit!(model, train_input, train_output)
println(score(model,test_input, test_output))

0.9321345218344909


In [39]:
model = SVC()
fit!(model, train_input, train_output)

LoadError: PyError ($(Expr(:escape, :(ccall(#= /Users/alexdopico/.julia/packages/PyCall/ilqDX/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError("could not convert string to float: '203.205.158.53-10.42.0.151-80-49839-6'")
  File "/Users/alexdopico/.julia/conda/3/lib/python3.9/site-packages/sklearn/svm/_base.py", line 192, in fit
    X, y = self._validate_data(
  File "/Users/alexdopico/.julia/conda/3/lib/python3.9/site-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/Users/alexdopico/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/validation.py", line 1106, in check_X_y
    X = check_array(
  File "/Users/alexdopico/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/validation.py", line 879, in check_array
    array = _asarray_with_order(array, order=order, dtype=dtype, xp=xp)
  File "/Users/alexdopico/.julia/conda/3/lib/python3.9/site